In [ ]:
import numpy as np
import pandas as pd

In [ ]:
bus_groups_time_series = pd.read_csv("../data/bus_groups_timeseries.csv", index_col=0)
bus_groups_time_series

In [ ]:
# ①5000kmごとにbinの番号をつける。
def calculate_bin_or_keep_nan(val):
    """
    valがNaNであれば、NaNを返します。
    そうでなければ、valを5000kmごとのビンに分類し、そのビンの番号を返します。
    ただし、ビンの番号が21以上の場合は21を返します。
    """
    if pd.isna(val):
        return np.nan
    else:
        bin_number = np.floor(val / 5000).astype(int) + 1
        return min(bin_number, 21)


numbers = bus_groups_time_series.map(calculate_bin_or_keep_nan)
numbers

In [ ]:
# 変化のカウント用の辞書を初期化
counts = {"x1": 0, "x2": 0, "x3": 0, "x4": 0}

# 各列(bus_name)に対してループ
for column in numbers.columns:
    # a4の条件は特別に扱う（20から21への変化）
    counts["x4"] += ((numbers[column].shift(-1) == 21) & (numbers[column] == 20)).sum()
    # 21の値を持つ行をNaNに置き換え
    modified_column = numbers[column].where(numbers[column] != 21)
    diff = modified_column.diff()
    # 変化のカウント
    counts["x1"] += (diff == 0).sum()
    counts["x2"] += (diff == 1).sum()
    counts["x3"] += (diff == 2).sum()

counts

In [ ]:
# ④kappa1とkappa2を計算する。dfの形で出力する。
def kappa1(x1, x2, x3, x4):
    return (x2 * (x2 + x3 + x4)) / ((x2 + x3) * (x1 + x2 + x3 + x4))


def kappa2(x1, x2, x3, x4):
    return (x3 * (x2 + x3 + x4)) / ((x2 + x3) * (x1 + x2 + x3 + x4))


kappa1 = kappa1(counts["x1"], counts["x2"], counts["x3"], counts["x4"])
kappa2 = kappa2(counts["x1"], counts["x2"], counts["x3"], counts["x4"])

kappa1, kappa2